Отриману гармоніку з накладеним на неї шумом відфільтруйте за допомогою фільтру на ваш вибір (наприклад scipy.signal.iirfilter, повний список за посиланням: https://docs.scipy.org/doc/scipy/reference/signal.html). Відфільтрована гармоніка має бути максимально близька до «чистої»

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from ipywidgets import interactive, FloatSlider, Checkbox, VBox

In [2]:
# Ініціалізуємо змінну для зберігання шуму
noise_cache = None
prev_noise_mean = None
prev_noise_std = None

def generate_harmonic(amplitude, frequency, phase, num_samples=1000):
    x = np.linspace(0, 2 * np.pi, num_samples)
    y = amplitude * np.sin(frequency * x + phase)
    return x, y

def add_noise(signal, mean=0, std=0.1):
    global noise_cache, prev_noise_mean, prev_noise_std
    if noise_cache is None or mean != prev_noise_mean or std != prev_noise_std:
        noise_cache = np.random.normal(mean, std, len(signal))
        prev_noise_mean = mean
        prev_noise_std = std
    return signal + noise_cache

t = np.linspace(0, 1, 1000)

Відобразіть відфільтровану «чисту» гармоніку поряд з початковою

In [3]:
def butterworth_filter(signal, cutoff_frequency, fs, order=5):
    # Визначаємо частоти діапазону фільтрації
    nyquist = 0.5 * fs
    normal_cutoff = cutoff_frequency / nyquist

    # Створюємо коефіцієнти для фільтру Баттерворта
    b, a = butter(order, normal_cutoff, btype='low', analog=False)

    # Застосовуємо фільтр до сигналу
    filtered_signal = filtfilt(b, a, signal)

    return filtered_signal

def plot_butterworth_filter(amplitude, frequency, phase, noise_mean, noise_std, show_filtered, cutoff_frequency):
    x, y = generate_harmonic(amplitude, frequency, phase)
    noisy_signal = add_noise(y, mean=noise_mean, std=noise_std)

    fs = 1000  # Припустимо, що частота дискретизації дорівнює 1000 Гц
    filtered_signal = butterworth_filter(noisy_signal, cutoff_frequency, fs)

    plt.figure(figsize=(12, 8))
    plt.plot(x, noisy_signal, label='Гармоніка з шумом', alpha=0.5)
    plt.plot(x, y, label='Гармоніка чиста', alpha=0.5)
    if show_filtered:
        plt.plot(x, filtered_signal, label=f'Фільтр Баттерворта (cutoff={cutoff_frequency})', linestyle='--')
    plt.xlabel('Час')
    plt.ylabel('Амплітуда')
    plt.title('Гармоніка з шумом та фільтр Баттерворта')
    plt.legend()
    plt.grid(True)
    plt.show()

Додайте відповідні інтерактивні елементи (чекбокс показу, параметри фільтру тощо) та оновіть існуючі: відфільтрована гармоніка має оновлюватись разом з початковою.

In [4]:
# Створення інтерактивних віджетів
amplitude_slider = FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0, description='Амплітуда:')
frequency_slider = FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0, description='Частота:')
phase_slider = FloatSlider(min=0, max=2*np.pi, step=0.1, value=0, description='Фаза:')
noise_mean_slider = FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.0, description='Середнє значення шуму:')
noise_std_slider = FloatSlider(min=0.1, max=2.0, step=0.1, value=0.1, description='Стандартне відхилення шуму:')
show_filtered_checkbox = Checkbox(value=False, description='Показати відфільтровану гармоніку')
cutoff_frequency_slider = FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0, description='Частота відсічки фільтра Баттерворта:')


# Створення інтерактивного віджету
interactive_plot = interactive(plot_butterworth_filter,
                               amplitude=amplitude_slider,
                               frequency=frequency_slider,
                               phase=phase_slider,
                               noise_mean=noise_mean_slider,
                               noise_std=noise_std_slider,
                               show_filtered=show_filtered_checkbox,
                               cutoff_frequency=cutoff_frequency_slider)

# Відображення віджету
VBox([interactive_plot])